In [6]:
import pandas as pd
import re
from functools import reduce
from glob import glob
import seaborn as sns

pd.options.display.max_rows = 999
pd.options.display.max_columns=100

In [7]:
def TDLog(file):
    List1=[]
    with open(file, 'r') as f:
        pat = '^\"(.+?)\" #[0-9]+.*nid=.+? (.+?) \[.*$'
        r = re.compile(pat)
        lines = f.readlines()
        line1=None
        for data in lines:
            if("tid=0x" in data):
                #print(data)
                line1 = re.search(pat,data)
                if line1 is not None:
                    List1.append([line1.group(1),line1.group(2)])
    columns=['thread','state']
    print(f"{file.split('/')[-1]} Total Threads = {len(List1)}")
    df = pd.DataFrame(List1)
    df.columns=columns
    df['tn'] = df['thread'].str.split('-',expand=True)[0]
    return df

def merged(dflist):
    data_frames=[]
    columns = ['thread','state']+[f"td{i}" for i in range(1,len(dflist)+1)]
    for df in dflist:
        data_frames.append(df.groupby(['tn','state']).count().sort_values(('thread'),ascending=False).reset_index())
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tn','state'],
                                                how='outer'), data_frames)
    df_merged.columns=columns
    return df_merged

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [15]:
directory = 'tmp/*'
LIST = []
for file in sorted_nicely(glob(directory)):
    LIST.append(TDLog(file))
cm =sns.cubehelix_palette(8, start=2, rot=0, dark=0.4, light=.95, reverse=False,as_cmap=True)
FINAL = merged(LIST).sort_values(['thread','td1'],ascending=[True,False]).reset_index(drop=True)

FINAL.style.background_gradient(cmap=cm,vmin=FINAL.min()[2], vmax=FINAL.max()[2]).highlight_null('grey')

td1 Total Threads = 60
td2 Total Threads = 60
td3 Total Threads = 60


,thread,state,td1,td2,td3
0,AWT,in Object.wait(),1,1,1
1,AWT,runnable,1,1,1
2,AWT,waiting on condition,1,1,1
3,AWTThreading pool,waiting on condition,1,1,1
4,ApplicationImpl pooled thread 138,waiting on condition,1,1,1
5,ApplicationImpl pooled thread 140,waiting on condition,1,1,1
6,ApplicationImpl pooled thread 141,waiting on condition,1,1,1
7,ApplicationImpl pooled thread 142,waiting on condition,1,1,1
8,ApplicationImpl pooled thread 143,waiting on condition,1,1,1
9,ApplicationImpl pooled thread 144,waiting on condition,1,1,1
